In [ ]:
# 3_visualization.ipynb
#
# This notebook generates interactive visualizations and creates
# a comprehensive HTML report of the ROM results:
# - Loads ROM results from MinIO
# - Creates interactive visualizations with Plotly
# - Generates a mode explorer
# - Creates error analysis visualizations
# - Compiles results into an HTML report

import os
import numpy as np
import matplotlib.pyplot as plt
import json
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import base64
import io
from datetime import datetime
import time
import tempfile
import boto3
from botocore.client import Config
import pandas as pd
import plotly.io as pio

# Configure plotly to generate static images at higher resolution
pio.templates.default = "plotly_white"

# Create a temporary local directory for processing
temp_dir = tempfile.mkdtemp()
print(f"Using temporary directory: {temp_dir}")

# Connect to MinIO
print("Connecting to MinIO...")
s3_endpoint = os.environ.get('S3_ENDPOINT', 'http://minio:9000')

# Fix the endpoint URL if the protocol is missing
if s3_endpoint and not s3_endpoint.startswith(('http://', 'https://')):
    s3_endpoint = f"http://{s3_endpoint}"
    print(f"Adding http:// prefix to endpoint: {s3_endpoint}")

s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID', 'minioadmin')
s3_secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY', 'minioadmin')

s3 = boto3.client('s3',
                  endpoint_url=s3_endpoint,
                  aws_access_key_id=s3_access_key,
                  aws_secret_access_key=s3_secret_key,
                  config=Config(signature_version='s3v4'))

# Define MinIO parameters
MINIO_BUCKET = 'rom-data'
MINIO_OUTPUT_PREFIX = 'rom-pipeline/outputs'

# Verify the previous step completed
try:
    marker_key = f"{MINIO_OUTPUT_PREFIX}/rom_modeling_completed.txt"
    s3.head_object(Bucket=MINIO_BUCKET, Key=marker_key)
    print("Previous step (ROM modeling) completed successfully")
except Exception as e:
    print(f"Warning: Previous step completion marker not found: {str(e)}")
    print("Continuing anyway...")

# Load parameters and metadata from MinIO
try:
    # Download params.json
    params_key = f"{MINIO_OUTPUT_PREFIX}/params.json"
    params_path = os.path.join(temp_dir, 'params.json')
    s3.download_file(MINIO_BUCKET, params_key, params_path)
    
    with open(params_path, 'r') as f:
        params = json.load(f)
    
    # Download preprocessing metadata
    preproc_key = f"{MINIO_OUTPUT_PREFIX}/preprocessed/metadata.json"
    preproc_path = os.path.join(temp_dir, 'preproc_metadata.json')
    s3.download_file(MINIO_BUCKET, preproc_key, preproc_path)
    
    with open(preproc_path, 'r') as f:
        preproc_metadata = json.load(f)
    
    # Download ROM metadata
    rom_key = f"{MINIO_OUTPUT_PREFIX}/rom/rom_metadata.json"
    rom_path = os.path.join(temp_dir, 'rom_metadata.json')
    s3.download_file(MINIO_BUCKET, rom_key, rom_path)
    
    with open(rom_path, 'r') as f:
        rom_metadata = json.load(f)
        
    print("Successfully loaded all metadata")
except Exception as e:
    print(f"Error loading metadata: {str(e)}")
    raise

# Update parameters for this step
params.update({
    "visualization_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "visualization_options": {
        "create_interactive_plots": True,
        "create_html_report": True
    }
})

# Extract metadata
n_snapshots = preproc_metadata['n_snapshots']
n_points = preproc_metadata['n_points']
n_dims = preproc_metadata['n_dims']
structured_grid = preproc_metadata['structured_grid']
n_modes_kept = rom_metadata['n_modes_kept']

if structured_grid and 'grid_shape' in preproc_metadata:
    grid_shape = tuple(preproc_metadata['grid_shape'])
    print(f"Grid shape: {grid_shape}")

# Load ROM results from MinIO
print("Loading ROM results from MinIO...")

# Download modes.npy
modes_key = f"{MINIO_OUTPUT_PREFIX}/rom/modes.npy"
modes_path = os.path.join(temp_dir, 'modes.npy')
s3.download_file(MINIO_BUCKET, modes_key, modes_path)
modes = np.load(modes_path)

# Download singular_values.npy
sv_key = f"{MINIO_OUTPUT_PREFIX}/rom/singular_values.npy"
sv_path = os.path.join(temp_dir, 'singular_values.npy')
s3.download_file(MINIO_BUCKET, sv_key, sv_path)
singular_values = np.load(sv_path)

# Download temporal_coefficients.npy
tc_key = f"{MINIO_OUTPUT_PREFIX}/rom/temporal_coefficients.npy"
tc_path = os.path.join(temp_dir, 'temporal_coefficients.npy')
s3.download_file(MINIO_BUCKET, tc_key, tc_path)
temporal_coeffs = np.load(tc_path)

# Optional: Download reconstructed_snapshots.npy if needed
try:
    rs_key = f"{MINIO_OUTPUT_PREFIX}/rom/reconstructed_snapshots.npy"
    rs_path = os.path.join(temp_dir, 'reconstructed_snapshots.npy')
    s3.download_file(MINIO_BUCKET, rs_key, rs_path)
    reconstructed_snapshots = np.load(rs_path)
    print("Loaded reconstructed snapshots")
except Exception as e:
    print(f"Note: Could not load reconstructed snapshots: {str(e)}")
    reconstructed_snapshots = None

# We also need mean_flow for some visualizations
try:
    mf_key = f"{MINIO_OUTPUT_PREFIX}/preprocessed/mean_flow.npy"
    mf_path = os.path.join(temp_dir, 'mean_flow.npy')
    s3.download_file(MINIO_BUCKET, mf_key, mf_path)
    mean_flow = np.load(mf_path)
    print("Loaded mean flow")
except Exception as e:
    print(f"Note: Could not load mean flow: {str(e)}")
    mean_flow = None

# Load snapshot_matrix if available for error analysis
try:
    sm_key = f"{MINIO_OUTPUT_PREFIX}/preprocessed/snapshot_matrix.npy"
    sm_path = os.path.join(temp_dir, 'snapshot_matrix.npy')
    s3.download_file(MINIO_BUCKET, sm_key, sm_path)
    snapshot_matrix = np.load(sm_path)
    print("Loaded snapshot matrix")
except Exception as e:
    print(f"Note: Could not load snapshot matrix: {str(e)}")
    snapshot_matrix = None

print(f"Loaded ROM data:")
print(f"  - Modes shape: {modes.shape}")
print(f"  - Singular values shape: {singular_values.shape}")
print(f"  - Temporal coefficients shape: {temporal_coeffs.shape}")

# Create interactive energy distribution visualization
print("Creating interactive energy plots...")

# Calculate energy content
energy = singular_values**2
total_energy = np.sum(energy)
energy_ratio = energy / total_energy
cumulative_energy = np.cumsum(energy_ratio)

# Create DataFrame for plotting
energy_df = pd.DataFrame({
    'Mode': np.arange(1, len(singular_values) + 1),
    'Energy Fraction': energy_ratio,
    'Cumulative Energy': cumulative_energy,
    'Singular Value': singular_values
})

# Interactive Energy Distribution
fig_energy = make_subplots(rows=1, cols=2,
                          subplot_titles=("Energy Distribution", "Cumulative Energy"),
                          specs=[[{"type": "scatter"}, {"type": "scatter"}]])

# Energy distribution (log scale)
fig_energy.add_trace(
    go.Scatter(x=energy_df['Mode'].values[:20], 
              y=energy_df['Energy Fraction'].values[:20],
              mode='lines+markers',
              name='Energy Fraction',
              marker=dict(size=8, color='blue')),
    row=1, col=1
)

# Cumulative energy
fig_energy.add_trace(
    go.Scatter(x=energy_df['Mode'].values[:20], 
              y=energy_df['Cumulative Energy'].values[:20],
              mode='lines+markers',
              name='Cumulative Energy',
              marker=dict(size=8, color='red')),
    row=1, col=2
)

# Add threshold line
fig_energy.add_trace(
    go.Scatter(x=[1, 20], 
              y=[rom_metadata['energy_threshold'], rom_metadata['energy_threshold']],
              mode='lines',
              name=f"{rom_metadata['energy_threshold']*100:.0f}% Threshold",
              line=dict(color='black', dash='dash')),
    row=1, col=2
)

# Add selected modes line
fig_energy.add_trace(
    go.Scatter(x=[n_modes_kept, n_modes_kept], 
              y=[0, 1],
              mode='lines',
              name=f"{n_modes_kept} Modes Selected",
              line=dict(color='green', dash='dash')),
    row=1, col=2
)

fig_energy.update_xaxes(title_text="Mode Number", row=1, col=1)
fig_energy.update_xaxes(title_text="Mode Number", row=1, col=2)
fig_energy.update_yaxes(title_text="Energy Fraction", type="log", row=1, col=1)
fig_energy.update_yaxes(title_text="Cumulative Energy", row=1, col=2)

fig_energy.update_layout(
    title="POD Energy Analysis",
    height=500,
    width=1000,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

# Save figure as HTML
energy_html_path = os.path.join(temp_dir, 'energy_distribution_interactive.html')
fig_energy.write_html(energy_html_path)

# Upload to MinIO
s3.upload_file(
    energy_html_path,
    MINIO_BUCKET,
    f"{MINIO_OUTPUT_PREFIX}/visualization/energy_distribution_interactive.html"
)
print(f"Interactive energy plot uploaded to {MINIO_BUCKET}/{MINIO_OUTPUT_PREFIX}/visualization/energy_distribution_interactive.html")

# Create temporal coefficients visualization
print("Creating temporal coefficients plot...")

# Prepare data for plotting
coeff_data = []
for i in range(min(5, n_modes_kept)):
    coeff_data.append(
        go.Scatter(
            x=np.arange(n_snapshots),
            y=temporal_coeffs[i, :],
            mode='lines+markers',
            name=f'Mode {i+1}',
            marker=dict(size=6)
        )
    )

fig_coeff = go.Figure(data=coeff_data)
fig_coeff.update_layout(
    title="Temporal Evolution of POD Coefficients",
    xaxis_title="Snapshot Index",
    yaxis_title="Coefficient Value",
    legend_title="Mode",
    height=500,
    width=1000
)

# Save to HTML
coeff_html_path = os.path.join(temp_dir, 'temporal_coefficients_interactive.html')
fig_coeff.write_html(coeff_html_path)

# Upload to MinIO
s3.upload_file(
    coeff_html_path,
    MINIO_BUCKET,
    f"{MINIO_OUTPUT_PREFIX}/visualization/temporal_coefficients_interactive.html"
)
print(f"Interactive coefficients plot uploaded to {MINIO_BUCKET}/{MINIO_OUTPUT_PREFIX}/visualization/temporal_coefficients_interactive.html")

# Create reconstruction error visualization
print("Creating reconstruction error visualization...")

# Get error data from ROM metadata
error_per_snapshot = np.array(rom_metadata['error_stats']['error_per_snapshot'])
mean_error = rom_metadata['error_stats']['mean_snapshot_error']

fig_error = go.Figure()
fig_error.add_trace(
    go.Scatter(
        x=np.arange(n_snapshots),
        y=error_per_snapshot,
        mode='lines+markers',
        name='Snapshot Error',
        marker=dict(size=6, color='blue')
    )
)
fig_error.add_trace(
    go.Scatter(
        x=[0, n_snapshots-1],
        y=[mean_error, mean_error],
        mode='lines',
        name=f'Mean Error: {mean_error:.6f}',
        line=dict(color='red', dash='dash')
    )
)

fig_error.update_layout(
    title=f"Reconstruction Error with {n_modes_kept} Modes",
    xaxis_title="Snapshot Index",
    yaxis_title="Relative Error",
    height=500,
    width=1000
)

# Save to HTML
error_html_path = os.path.join(temp_dir, 'reconstruction_error_interactive.html')
fig_error.write_html(error_html_path)

# Upload to MinIO
s3.upload_file(
    error_html_path,
    MINIO_BUCKET,
    f"{MINIO_OUTPUT_PREFIX}/visualization/reconstruction_error_interactive.html"
)
print(f"Interactive error plot uploaded to {MINIO_BUCKET}/{MINIO_OUTPUT_PREFIX}/visualization/reconstruction_error_interactive.html")

# If we have structured grid data, create mode explorer visualizations
if structured_grid:
    print("Creating mode explorer visualizations...")
    
    # Create visualizations for the first few spatial modes
    n_modes_to_show = min(6, n_modes_kept)
    
    # Create a combined visualization of the first 4 modes
    n_compare = min(4, n_modes_kept)
    fig_modes = make_subplots(rows=2, cols=2, 
                             subplot_titles=[f"Mode {i+1}" for i in range(n_compare)])
    
    for i in range(n_compare):
        row, col = i // 2 + 1, i % 2 + 1
        
        if n_dims == 1:
            # Scalar field
            mode_reshaped = modes[:, i].reshape(grid_shape)
            
            fig_modes.add_trace(
                go.Heatmap(
                    z=mode_reshaped,
                    colorscale='RdBu_r',
                    zmid=0
                ),
                row=row, col=col
            )
        else:
            # Vector field - magnitude
            mode_reshaped = modes[:, i].reshape(grid_shape[0], grid_shape[1], n_dims)
            mode_magnitude = np.sqrt(np.sum(mode_reshaped**2, axis=2))
            
            fig_modes.add_trace(
                go.Heatmap(
                    z=mode_magnitude,
                    colorscale='Viridis'
                ),
                row=row, col=col
            )
    
    fig_modes.update_layout(
        title="POD Mode Comparison",
        height=900,
        width=1000
    )
    
    # Save to HTML
    modes_html_path = os.path.join(temp_dir, 'modes_comparison_interactive.html')
    fig_modes.write_html(modes_html_path)
    
    # Upload to MinIO
    s3.upload_file(
        modes_html_path,
        MINIO_BUCKET,
        f"{MINIO_OUTPUT_PREFIX}/visualization/modes_comparison_interactive.html"
    )
    print(f"Interactive modes comparison uploaded to {MINIO_BUCKET}/{MINIO_OUTPUT_PREFIX}/visualization/modes_comparison_interactive.html")

# Create an HTML report
print("Creating HTML report...")

# Get existing visualizations from MinIO
report_dir = os.path.join(temp_dir, 'report')
os.makedirs(report_dir, exist_ok=True)

# Download existing visualizations
try:
    viz_prefix = f"{MINIO_OUTPUT_PREFIX}/visualizations/"
    response = s3.list_objects_v2(Bucket=MINIO_BUCKET, Prefix=viz_prefix)
    
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'].endswith(('.png', '.jpg', '.jpeg', '.svg')):
                filename = os.path.basename(obj['Key'])
                local_path = os.path.join(report_dir, filename)
                
                print(f"  Downloading {obj['Key']} to {local_path}")
                s3.download_file(MINIO_BUCKET, obj['Key'], local_path)
except Exception as e:
    print(f"Warning: Could not download existing visualizations: {str(e)}")

# Create HTML report with visualizations and interactive elements
html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Reduced Order Modeling (ROM) Results</title>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
        body {{
            font-family: Arial, sans-serif;
            line-height: 1.6;
            margin: 0;
            padding: 0;
            color: #333;
        }}
        .container {{
            width: 90%;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
        }}
        header {{
            background-color: #3498db;
            color: white;
            padding: 1rem;
            text-align: center;
            margin-bottom: 2rem;
        }}
        h1, h2, h3 {{
            color: #2c3e50;
        }}
        .section {{
            margin-bottom: 2rem;
            padding: 1rem;
            background-color: #f9f9f9;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }}
        .metrics {{
            display: flex;
            flex-wrap: wrap;
            justify-content: space-between;
        }}
        .metric-card {{
            flex: 1;
            min-width: 200px;
            margin: 10px;
            padding: 15px;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            background-color: white;
            text-align: center;
        }}
        .metric-value {{
            font-size: 1.8em;
            font-weight: bold;
            color: #3498db;
        }}
        img {{
            max-width: 100%;
            height: auto;
            display: block;
            margin: 1rem auto;
            border: 1px solid #ddd;
            border-radius: 5px;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 1rem 0;
        }}
        table, th, td {{
            border: 1px solid #ddd;
        }}
        th, td {{
            padding: 8px;
            text-align: left;
        }}
        th {{
            background-color: #f2f2f2;
        }}
    </style>
</head>
<body>
    <header>
        <h1>Reduced Order Modeling (ROM) Results</h1>
        <p>Cylinder Flow Dataset - POD Analysis</p>
        <p>Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</p>
    </header>
    
    <div class="container">
        <div class="section">
            <h2>1. Overview</h2>
            <p>This report presents the results of Proper Orthogonal Decomposition (POD) applied to the Cylinder Flow Dataset.</p>
            
            <div class="metrics">
                <div class="metric-card">
                    <h3>Dataset</h3>
                    <div class="metric-value">Cylinder Flow</div>
                    <p>Fluid flow past a cylinder</p>
                </div>
                <div class="metric-card">
                    <h3>Snapshots</h3>
                    <div class="metric-value">{n_snapshots}</div>
                    <p>Number of time steps</p>
                </div>
                <div class="metric-card">
                    <h3>POD Modes</h3>
                    <div class="metric-value">{n_modes_kept}</div>
                    <p>Reduced basis size</p>
                </div>
                <div class="metric-card">
                    <h3>Energy Captured</h3>
                    <div class="metric-value">{rom_metadata['energy_stats']['energy_captured'] * 100:.2f}%</div>
                    <p>Of total energy</p>
                </div>
            </div>
        </div>
        
        <div class="section">
            <h2>2. Energy Analysis</h2>
            <p>Analysis of energy distribution across POD modes.</p>
            
            <img src="energy_distribution.png" alt="Energy Distribution" />
            
            <div class="subsection">
                <h3>Energy Distribution Table</h3>
                <table>
                    <tr>
                        <th>Mode</th>
                        <th>Singular Value</th>
                        <th>Energy Fraction (%)</th>
                        <th>Cumulative Energy (%)</th>
                    </tr>
"""

# Add rows for first 10 modes
for i in range(min(10, n_modes_kept)):
    html_template += f"""
                    <tr>
                        <td>{i+1}</td>
                        <td>{singular_values[i]:.6f}</td>
                        <td>{energy_ratio[i] * 100:.6f}%</td>
                        <td>{cumulative_energy[i] * 100:.6f}%</td>
                    </tr>
    """

html_template += f"""
                </table>
            </div>
        </div>
        
        <div class="section">
            <h2>3. Temporal Coefficients</h2>
            <p>Evolution of POD mode coefficients over time.</p>
            
            <img src="temporal_coefficients.png" alt="Temporal Coefficients" />
        </div>
        
        <div class="section">
            <h2>4. Reconstruction Error</h2>
            <p>Analysis of reconstruction error using {n_modes_kept} POD modes.</p>
            
            <div class="metrics">
                <div class="metric-card">
                    <h3>Overall Error</h3>
                    <div class="metric-value">{rom_metadata['error_stats']['overall_relative_error']:.6f}</div>
                    <p>Relative error norm</p>
                </div>
                <div class="metric-card">
                    <h3>Mean Snapshot Error</h3>
                    <div class="metric-value">{rom_metadata['error_stats']['mean_snapshot_error']:.6f}</div>
                    <p>Average error across snapshots</p>
                </div>
                <div class="metric-card">
                    <h3>Max Snapshot Error</h3>
                    <div class="metric-value">{rom_metadata['error_stats']['max_snapshot_error']:.6f}</div>
                    <p>Maximum error in any snapshot</p>
                </div>
            </div>
            
            <img src="reconstruction_error.png" alt="Reconstruction Error" />
        </div>
"""

# Add spatial mode visualization section if applicable
if structured_grid:
    html_template += f"""
        <div class="section">
            <h2>5. Spatial Modes</h2>
            <p>Visualization of the dominant spatial POD modes.</p>
            
            <img src="spatial_modes.png" alt="Spatial Modes" />
            
            <div class="subsection">
                <h3>Reconstruction Comparison</h3>
                <p>Comparison between an original snapshot and its POD reconstruction.</p>
                
                <img src="reconstruction_comparison.png" alt="Reconstruction Comparison" />
            </div>
        </div>
    """

# Complete the HTML
html_template += f"""
        <div class="section">
            <h2>6. Conclusion</h2>
            <p>The Proper Orthogonal Decomposition (POD) analysis successfully reduced the dimensionality of the Cylinder Flow Dataset from {n_points * n_dims} degrees of freedom to {n_modes_kept} modes, while capturing {rom_metadata['energy_stats']['energy_captured'] * 100:.2f}% of the system's energy.</p>
            
            <p>The reconstruction achieved an overall relative error of {rom_metadata['error_stats']['overall_relative_error']:.6f}, demonstrating that the POD-based reduced-order model effectively captures the dominant flow structures.</p>
            
            <p>This analysis confirms that POD is an effective technique for creating reduced-order models of the cylinder flow, providing a compact representation that preserves the essential dynamics of the system.</p>
        </div>
    </div>
    
    <footer style="background-color: #34495e; color: white; text-align: center; padding: 1rem; margin-top: 2rem;">
        <p>Generated by ROM Pipeline | Elyra + Apache Airflow</p>
    </footer>
</body>
</html>
"""

# Write HTML report to file
report_path = os.path.join(temp_dir, 'rom_report.html')
with open(report_path, 'w') as f:
    f.write(html_template)

# Upload to MinIO
s3.upload_file(
    report_path,
    MINIO_BUCKET,
    f"{MINIO_OUTPUT_PREFIX}/visualization/rom_report.html"
)

print(f"HTML report uploaded to {MINIO_BUCKET}/{MINIO_OUTPUT_PREFIX}/visualization/rom_report.html")

# Upload completion marker
with open(os.path.join(temp_dir, 'visualization_completed.txt'), 'w') as f:
    f.write("Visualization completed successfully")

s3.upload_file(
    os.path.join(temp_dir, 'visualization_completed.txt'),
    MINIO_BUCKET, 
    f"{MINIO_OUTPUT_PREFIX}/visualization_completed.txt"
)

print("\nVisualization completed successfully!")
print(f"All results have been uploaded to {MINIO_BUCKET}/{MINIO_OUTPUT_PREFIX}/")